[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/microsoft/TextWorld/blob/main/notebooks/Building%20a%20simple%20agent.ipynb)

# Training LLM to play games

This tutorial outlines the steps to build an agent that learns how to play __choice-based__ text-based games generated with TextWorld.

### Prerequisite
Install TextWorld as described in the [README.md](https://github.com/microsoft/TextWorld#readme). Most of the time, a simple `pip install` should work.

In [1]:
!pip install textworld
!pip install gymnasium
!pip install gym

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 18.0 MB/s eta 0:00:00


and [PyTorch](https://pytorch.org/) (tested with both v1.8.2 and v.1.9.0).

In [2]:
!pip install torch torchvision torchaudio

#Extra

In [3]:
!pip install transformers datasets accelerate

**[Optional]** Download all data beforehand. Otherwise, they are going to be generated as needed (slower).

In [4]:
!wget https://aka.ms/textworld/notebooks/data.zip
!unzip -nq data.zip && rm -f data.zip

--2025-11-25 12:40:17--  https://aka.ms/textworld/notebooks/data.zip
Resolving aka.ms (aka.ms)... 23.204.250.75
Connecting to aka.ms (aka.ms)|23.204.250.75|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://textworld.blob.core.windows.net/$web/notebooks/data.zip [following]
--2025-11-25 12:40:17--  https://textworld.blob.core.windows.net/$web/notebooks/data.zip
Resolving textworld.blob.core.windows.net (textworld.blob.core.windows.net)... 57.150.159.97
Connecting to textworld.blob.core.windows.net (textworld.blob.core.windows.net)|57.150.159.97|:443... connected.
HTTP request sent, awaiting response... 409 Public access is not permitted on this storage account.
2025-11-25 12:40:17 ERROR 409: Public access is not permitted on this storage account..

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


## Learning challenges
Training an agent such that it can learn how to play text-based games is not trivial. Among other challenges, we have to deal with

1. a combinatorial action space (that grows w.r.t. vocabulary)
2. a really sparse reward signal.

To ease the learning process, we will be requesting additional information alongside the game's narrative (as covered in [Playing TextWorld generated games with OpenAI Gym](Playing%20TextWorld%20generated%20games%20with%20OpenAI%20Gym.ipynb#Interact-with-the-game)). More specifically, we will request the following information:

- __Description__:
For every game state, we will get the output of the `look` command which describes the current location;

- __Inventory__:
For every game state, we will get the output of the `inventory` command which describes the player's inventory;

- __Admissible commands__:
For every game state, we will get the list of commands guaranteed to be understood by the game interpreter;

- __Intermediate reward__:
For every game state, we will get an intermediate reward which can either be:
  - __-1__: last action needs to be undone before resuming the quest
  -  __0__: last action didn't affect the quest
  -  __1__: last action brought us closer to completing the quest

- __Entities__:
For every game, we will get a list of entity names that the agent can interact with.


## Simple test games
We can use TextWorld to generate a few simple games with the following handcrafted world
```
                     Bathroom
                        +
                        |
                        +
    Bedroom +-(d1)-+ Kitchen +--(d2)--+ Backyard
      (P)               +                  +
                        |                  |
                        +                  +
                   Living Room           Garden
```
where the goal is always to retrieve a hidden food item and put it on the stove which is located in the kitchen. One can lose the game if it eats the food item instead of putting it on the stove!

Using `tw-make tw-simple ...`, we are going to generate the following 7 games:

| gamefile | description |
| :------- | :---------- |
| `games/rewardsDense_goalDetailed.z8` | dense reward + detailed instructions |
| `games/rewardsBalanced_goalDetailed.z8` | balanced rewards + detailed instructions |
| `games/rewardsSparse_goalDetailed.z8` | sparse rewards + detailed instructions |
| | |
| `games/rewardsDense_goalBrief.z8` | dense rewards + no instructions but the goal is mentionned |
| `games/rewardsBalanced_goalBrief.z8` | balanced rewards + no instructions but the goal is mentionned |
| `games/rewardsSparse_goalBrief.z8` | sparse rewards + no instructions but the goal is mentionned |
| | |
| `games/rewardsSparse_goalNone.z8` | sparse rewards + no instructions/goal<br>_Hint: there's an hidden note in the game that describes the goal!_ |

In [5]:
# You can skip this if you already downloaded the games in the prequisite section.

# Same as !make_games.sh
!tw-make tw-simple --rewards dense    --goal detailed --seed 18 --test --silent -f --output games/tw-rewardsDense_goalDetailed.z8
!tw-make tw-simple --rewards balanced --goal detailed --seed 18 --test --silent -f --output games/tw-rewardsBalanced_goalDetailed.z8
!tw-make tw-simple --rewards sparse   --goal detailed --seed 18 --test --silent -f --output games/tw-rewardsSparse_goalDetailed.z8
!tw-make tw-simple --rewards dense    --goal brief    --seed 18 --test --silent -f --output games/tw-rewardsDense_goalBrief.z8
!tw-make tw-simple --rewards balanced --goal brief    --seed 18 --test --silent -f --output games/tw-rewardsBalanced_goalBrief.z8
!tw-make tw-simple --rewards sparse   --goal brief    --seed 18 --test --silent -f --output games/tw-rewardsSparse_goalBrief.z8
!tw-make tw-simple --rewards sparse   --goal none     --seed 18 --test --silent -f --output games/tw-rewardsSparse_goalNone.z8

## Building the random baseline
Let's start with building an agent that simply selects an admissible command at random.

In [6]:
from typing import Mapping, Any

import numpy as np

import textworld.gym


class RandomAgent(textworld.gym.Agent):
    """ Agent that randomly selects a command from the admissible ones. """
    def __init__(self, seed=1234):
        self.seed = seed
        self.rng = np.random.RandomState(self.seed)

    @property
    def infos_to_request(self) -> textworld.EnvInfos:
        return textworld.EnvInfos(admissible_commands=True)

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:
        return self.rng.choice(infos["admissible_commands"])


## Play function
Let's write a simple play function that we will use to evaluate our agent on a given game.

In [7]:
import os
from glob import glob

import textworld.gym

import torch


def play(agent, path, max_step=100, nb_episodes=10, verbose=True):
    torch.manual_seed(20211021)  # For reproducibility when using action sampling.

    infos_to_request = agent.infos_to_request
    infos_to_request.max_score = True  # Needed to normalize the scores.

    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.z8"))

    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=infos_to_request,
                                          max_episode_steps=max_step)
    env = textworld.gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")

    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores = [], [], []
    for no_episode in range(nb_episodes):
        obs, infos = env.reset()  # Start new episode.

        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs, score, done, infos)
            obs, score, done, infos = env.step(command)
            nb_moves += 1

        agent.act(obs, score, done, infos)  # Let the agent know the game is done.

        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])

    env.close()
    if verbose:
        if os.path.isdir(path):
            msg = "  \tavg. steps: {:5.1f}; avg. normalized score: {:4.1f} / {}."
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
            print(msg.format(np.mean(avg_moves), np.mean(avg_scores), infos["max_score"]))


#### Evaluate the random agent

In [8]:
# We report the score and steps averaged over 10 playthroughs.
play(RandomAgent(), "./games/tw-rewardsDense_goalDetailed.z8")    # Dense rewards
play(RandomAgent(), "./games/tw-rewardsBalanced_goalDetailed.z8") # Balanced rewards
play(RandomAgent(), "./games/tw-rewardsSparse_goalDetailed.z8")   # Sparse rewards

tw-rewardsDense_goalDetailed.z8..........  	avg. steps: 100.0; avg. score:  4.2 / 10.
tw-rewardsBalanced_goalDetailed.z8..........  	avg. steps: 100.0; avg. score:  0.7 / 4.
tw-rewardsSparse_goalDetailed.z8..........  	avg. steps: 100.0; avg. score:  0.0 / 1.


## Neural agent

Now, let's create an agent that can learn to play text-based games. The agent will be trained to select a command from the list of admissible commands given the current game's narrative, inventory, and room description. Here is an overview of the architecture used for the agent:

<div>
  <img src="https://raw.githubusercontent.com/MarcCote/TextWorld/msr_summit_2021/notebooks/figs/neural_agent.png" width="500"/>
</div>



### Code
Here's the implementation of that learning agent built with [PyTorch](https://pytorch.org/).

In [9]:
import re
from typing import List, Mapping, Any, Optional
from collections import defaultdict

import numpy as np

import textworld
import textworld.gym
from textworld import EnvInfos

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class CommandScorer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CommandScorer, self).__init__()
        torch.manual_seed(42)  # For reproducibility
        self.embedding    = nn.Embedding(input_size, hidden_size)
        self.encoder_gru  = nn.GRU(hidden_size, hidden_size)
        self.cmd_encoder_gru  = nn.GRU(hidden_size, hidden_size)
        self.state_gru    = nn.GRU(hidden_size, hidden_size)
        self.hidden_size  = hidden_size
        self.state_hidden = torch.zeros(1, 1, hidden_size, device=device)
        self.critic       = nn.Linear(hidden_size, 1)
        self.att_cmd      = nn.Linear(hidden_size * 2, 1)

    def forward(self, obs, commands, **kwargs):
        input_length = obs.size(0)
        batch_size = obs.size(1)
        nb_cmds = commands.size(1)

        embedded = self.embedding(obs)
        encoder_output, encoder_hidden = self.encoder_gru(embedded)
        state_output, state_hidden = self.state_gru(encoder_hidden, self.state_hidden)
        self.state_hidden = state_hidden
        value = self.critic(state_output)

        # Attention network over the commands.
        cmds_embedding = self.embedding.forward(commands)
        _, cmds_encoding_last_states = self.cmd_encoder_gru.forward(cmds_embedding)  # 1 x cmds x hidden

        # Same observed state for all commands.
        cmd_selector_input = torch.stack([state_hidden] * nb_cmds, 2)  # 1 x batch x cmds x hidden

        # Same command choices for the whole batch.
        cmds_encoding_last_states = torch.stack([cmds_encoding_last_states] * batch_size, 1)  # 1 x batch x cmds x hidden

        # Concatenate the observed state and command encodings.
        cmd_selector_input = torch.cat([cmd_selector_input, cmds_encoding_last_states], dim=-1)

        # Compute one score per command.
        scores = F.relu(self.att_cmd(cmd_selector_input)).squeeze(-1)  # 1 x Batch x cmds

        probs = F.softmax(scores, dim=2)  # 1 x Batch x cmds
        index = probs[0].multinomial(num_samples=1).unsqueeze(0) # 1 x batch x indx
        return scores, index, value

    def reset_hidden(self, batch_size):
        self.state_hidden = torch.zeros(1, batch_size, self.hidden_size, device=device)


class NeuralAgent:
    """ Simple Neural Agent for playing TextWorld games. """
    MAX_VOCAB_SIZE = 1000
    UPDATE_FREQUENCY = 10
    LOG_FREQUENCY = 1000
    GAMMA = 0.9

    def __init__(self) -> None:
        self._initialized = False
        self._epsiode_has_started = False
        self.id2word = ["<PAD>", "<UNK>"]
        self.word2id = {w: i for i, w in enumerate(self.id2word)}

        self.model = CommandScorer(input_size=self.MAX_VOCAB_SIZE, hidden_size=128)
        self.optimizer = optim.Adam(self.model.parameters(), 0.00003)

        self.mode = "test"

    def train(self):
        self.mode = "train"
        self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}
        self.transitions = []
        self.model.reset_hidden(1)
        self.last_score = 0
        self.no_train_step = 0

    def test(self):
        self.mode = "test"
        self.model.reset_hidden(1)

    @property
    def infos_to_request(self) -> EnvInfos:
        return EnvInfos(description=True, inventory=True, admissible_commands=True,
                        won=True, lost=True)

    def _get_word_id(self, word):
        if word not in self.word2id:
            if len(self.word2id) >= self.MAX_VOCAB_SIZE:
                return self.word2id["<UNK>"]

            self.id2word.append(word)
            self.word2id[word] = len(self.word2id)

        return self.word2id[word]

    def _tokenize(self, text):
        # Simple tokenizer: strip out all non-alphabetic characters.
        text = re.sub("[^a-zA-Z0-9\- ]", " ", text)
        word_ids = list(map(self._get_word_id, text.split()))
        return word_ids

    def _process(self, texts):
        texts = list(map(self._tokenize, texts))
        max_len = max(len(l) for l in texts)
        padded = np.ones((len(texts), max_len)) * self.word2id["<PAD>"]

        for i, text in enumerate(texts):
            padded[i, :len(text)] = text

        padded_tensor = torch.from_numpy(padded).type(torch.long).to(device)
        padded_tensor = padded_tensor.permute(1, 0) # Batch x Seq => Seq x Batch
        return padded_tensor

    def _discount_rewards(self, last_values):
        returns, advantages = [], []
        R = last_values.data
        for t in reversed(range(len(self.transitions))):
            rewards, _, _, values = self.transitions[t]
            R = rewards + self.GAMMA * R
            adv = R - values
            returns.append(R)
            advantages.append(adv)

        return returns[::-1], advantages[::-1]

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> Optional[str]:

        # Build agent's observation: feedback + look + inventory.
        input_ = "{}\n{}\n{}".format(obs, infos["description"], infos["inventory"])

        # Tokenize and pad the input and the commands to chose from.
        input_tensor = self._process([input_])
        commands_tensor = self._process(infos["admissible_commands"])

        # Get our next action and value prediction.
        outputs, indexes, values = self.model(input_tensor, commands_tensor)
        action = infos["admissible_commands"][indexes[0]]

        if self.mode == "test":
            if done:
                self.model.reset_hidden(1)
            return action

        self.no_train_step += 1

        if self.transitions:
            reward = score - self.last_score  # Reward is the gain/loss in score.
            self.last_score = score
            if infos["won"]:
                reward += 100
            if infos["lost"]:
                reward -= 100

            self.transitions[-1][0] = reward  # Update reward information.

        self.stats["max"]["score"].append(score)
        if self.no_train_step % self.UPDATE_FREQUENCY == 0:
            # Update model
            returns, advantages = self._discount_rewards(values)

            loss = 0
            for transition, ret, advantage in zip(self.transitions, returns, advantages):
                reward, indexes_, outputs_, values_ = transition

                advantage        = advantage.detach() # Block gradients flow here.
                probs            = F.softmax(outputs_, dim=2)
                log_probs        = torch.log(probs)
                log_action_probs = log_probs.gather(2, indexes_)
                policy_loss      = (-log_action_probs * advantage).sum()
                value_loss       = (.5 * (values_ - ret) ** 2.).sum()
                entropy     = (-probs * log_probs).sum()
                loss += policy_loss + 0.5 * value_loss - 0.1 * entropy

                self.stats["mean"]["reward"].append(reward)
                self.stats["mean"]["policy"].append(policy_loss.item())
                self.stats["mean"]["value"].append(value_loss.item())
                self.stats["mean"]["entropy"].append(entropy.item())
                self.stats["mean"]["confidence"].append(torch.exp(log_action_probs).item())

            if self.no_train_step % self.LOG_FREQUENCY == 0:
                msg = "{:6d}. ".format(self.no_train_step)
                msg += "  ".join("{}: {: 3.3f}".format(k, np.mean(v)) for k, v in self.stats["mean"].items())
                msg += "  " + "  ".join("{}: {:2d}".format(k, np.max(v)) for k, v in self.stats["max"].items())
                msg += "  vocab: {:3d}".format(len(self.id2word))
                print(msg)
                self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}

            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 40)
            self.optimizer.step()
            self.optimizer.zero_grad()

            self.transitions = []
            self.model.reset_hidden(1)
        else:
            # Keep information about transitions for Truncated Backpropagation Through Time.
            self.transitions.append([None, indexes, outputs, values])  # Reward will be set on the next call

        if done:
            self.last_score = 0  # Will be starting a new episode. Reset the last score.

        return action

<>:114: SyntaxWarning: invalid escape sequence '\-'
<>:114: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipython-input-3165348506.py:114: SyntaxWarning: invalid escape sequence '\-'
  text = re.sub("[^a-zA-Z0-9\- ]", " ", text)


### Training the neural agent
Let's first evaluate the agent before training to get a sense of its initial performance.

In [11]:
agent = NeuralAgent()
agent.model.to(device) # Ensure the model is on the correct device
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")

tw-rewardsDense_goalDetailed.z8..........  	avg. steps: 100.0; avg. score:  5.0 / 10.


Unsurprisingly, the result is not much different from what the random agent can achieve since our neural agent is initialized to a random policy.

Let's train the agent for a few episodes.

In [12]:
from time import time
agent = NeuralAgent()
agent.model.to(device)

print("Training")
agent.train()  # Tell the agent it should update its parameters.
starttime = time()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8", nb_episodes=500, verbose=False)  # Dense rewards game.

print("Trained in {:.2f} secs".format(time() - starttime))

# Save the trained agent.
import os
os.makedirs('checkpoints', exist_ok=True)
torch.save(agent, 'checkpoints/agent_trained_on_single_game.pt')

Training
  1000. reward:  0.056  policy:  0.352  value:  0.095  entropy:  2.468  confidence:  0.089  score:  9  vocab: 301
  2000. reward:  0.040  policy:  0.082  value:  0.069  entropy:  2.333  confidence:  0.099  score:  6  vocab: 301
  3000. reward:  0.054  policy:  0.173  value:  0.092  entropy:  2.395  confidence:  0.095  score:  7  vocab: 301
  4000. reward:  0.041  policy: -0.022  value:  0.072  entropy:  2.363  confidence:  0.098  score:  6  vocab: 301
  5000. reward:  0.053  policy:  0.038  value:  0.098  entropy:  2.405  confidence:  0.096  score:  8  vocab: 301
  6000. reward:  0.048  policy:  0.055  value:  0.087  entropy:  2.388  confidence:  0.099  score:  7  vocab: 301
  7000. reward:  0.047  policy: -0.051  value:  0.066  entropy:  2.338  confidence:  0.104  score:  8  vocab: 301
  8000. reward:  0.052  policy:  0.084  value:  0.090  entropy:  2.425  confidence:  0.097  score:  7  vocab: 301
  9000. reward:  0.050  policy:  0.002  value:  0.074  entropy:  2.364  confide

#### Testing the agent trained on a single game

In [17]:
# We report the score and steps averaged over 10 playthroughs.
agent = torch.load('checkpoints/agent_trained_on_single_game.pt',weights_only=False)
agent.test()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")  # Dense rewards game.

tw-rewardsDense_goalDetailed.z8..........  	avg. steps:  77.7; avg. score:  8.7 / 10.


Of course, since we trained on that single simple game, it's not surprinsing the agent can achieve a high score on it. It would be more interesting to evaluate the generalization capability of the agent.

To do so, we are going to test the agent on another game drawn from the same game distribution (i.e. same world but the goal is to pick another food item). Let's generate `games/another_game.z8` with the same rewards density (`--rewards dense`) and the same goal description (`--goal detailed`), but using `--seed 1` and without the `--test` flag (to make sure the game is not part of the test set since `games/rewardsDense_goalDetailed.z8` is).

In [18]:
!tw-make tw-simple --rewards dense --goal detailed --seed 1 --output games/tw-another_game.z8 -v -f

Global seed: 1
Game generated: /content/games/tw-another_game.z8

Objective:
Hey, thanks for coming over to the TextWorld today, there is something I need you to do for me. First of all, you could, like, look and see that the antique trunk inside the bedroom is opened. Then, recover the old key from the antique trunk. Then, make absolutely sure that the wooden door inside the bedroom is unlocked. After unlocking the wooden door, open the wooden door in the bedroom. Then, try to head east. After that, try to travel south. Once you get through with that, take the milk from the couch within the living room. Having taken the milk, attempt to travel north. That done, rest the milk on the stove inside the kitchen. And if you do that, you're the winner!

Walkthrough:
open antique trunk > take old key from antique trunk > unlock wooden door with old key > open wooden door > go east > go south > take milk from couch > go north > put milk on stove

-= Stats =-
Nb. locations: 6
Nb. objects: 28


In [19]:
# We report the score and steps averaged over 10 playthroughs.
play(RandomAgent(), "./games/tw-another_game.z8")
play(agent, "./games/tw-another_game.z8")

tw-another_game.z8..........  	avg. steps: 100.0; avg. score:  3.9 / 8.
tw-another_game.z8..........  	avg. steps: 100.0; avg. score:  5.6 / 8.


As we can see the trained agent does a bit better than the random agent. In order to improve the agent's generalization capability, we should train it on many different games drawn from the game distribution.

One could use the following command to easily generate 100 training games:

In [20]:
# You can skip this if you already downloaded the data in the prequisite section.

! seq 1 100 | xargs -n1 -P4 tw-make tw-simple --rewards dense --goal detailed --format z8 --output training_games/ --seed

Global seed: 1
Global seed: 2
Global seed: 3
Global seed: 4
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-ek06H8B7uqoYFVEy.z8
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-E5eLHkaXFk6BSgR1.z8
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-ekDZtbGXIbO5FKp8.z8
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-D8gMTlO8cPoEtgZx.z8
Global seed: 6
Global seed: 7
Global seed: 5
Global seed: 8
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-7KpYUDDdckE0cBqZ.z8
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-KJODI168SvJVFM9x.z8
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-o2RVTmrEi6R5T3p0.z8
Game generated: /content/training_games/tw-simple-rDense+gDetailed+train-house-GP-68kvf8x7TBd9Iq0P.z8
Global seed: 9
Global seed: 10
Global seed: 11
Global seed: 12
G

Then, we train our agent on that set of training games.

In [21]:
from time import time
agent = NeuralAgent()
agent.model.to(device)

print("Training on 100 games")
agent.train()  # Tell the agent it should update its parameters.
starttime = time()
play(agent, "./training_games/", nb_episodes=100 * 5, verbose=False)  # Each game will be seen 5 times.
print("Trained in {:.2f} secs".format(time() - starttime))

# Save the trained agent.
import os
os.makedirs('checkpoints', exist_ok=True)
torch.save(agent, 'checkpoints/agent_trained_on_multiple_games.pt')

Training on 100 games
  1000. reward:  0.050  policy:  0.398  value:  0.102  entropy:  2.377  confidence:  0.096  score:  9  vocab: 448
  2000. reward:  0.051  policy:  0.238  value:  0.091  entropy:  2.361  confidence:  0.097  score:  9  vocab: 544
  3000. reward:  0.043  policy:  0.050  value:  0.075  entropy:  2.340  confidence:  0.099  score:  7  vocab: 601
  4000. reward:  0.042  policy:  0.031  value:  0.072  entropy:  2.352  confidence:  0.099  score:  6  vocab: 624
  5000. reward:  0.048  policy:  0.087  value:  0.101  entropy:  2.397  confidence:  0.095  score:  5  vocab: 629
  6000. reward:  0.054  policy:  0.093  value:  0.092  entropy:  2.420  confidence:  0.094  score:  8  vocab: 645
  7000. reward:  0.052  policy: -0.004  value:  0.089  entropy:  2.411  confidence:  0.095  score:  7  vocab: 658
  8000. reward:  0.051  policy:  0.033  value:  0.079  entropy:  2.401  confidence:  0.097  score:  7  vocab: 664
  9000. reward: -0.057  policy: -1.905  value:  24.984  entropy:  

#### Testing the agent trained on 100 games.

In [22]:
agent = torch.load('checkpoints/agent_trained_on_multiple_games.pt', weights_only = False)
agent.test()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")  # Averaged over 10 playthroughs.
play(agent, "./games/tw-another_game.z8")               # Averaged over 10 playthroughs.

tw-rewardsDense_goalDetailed.z8..........  	avg. steps:  83.6; avg. score:  9.0 / 10.
tw-another_game.z8..........  	avg. steps:  95.3; avg. score:  5.8 / 8.


Compare it to the agent trained on a single game.

In [24]:
agent = torch.load('checkpoints/agent_trained_on_single_game.pt',weights_only=False)
agent.test()
play(agent, "./games/tw-rewardsDense_goalDetailed.z8")  # Averaged over 10 playthroughs.
play(agent, "./games/tw-another_game.z8")               # Averaged over 10 playthroughs.

tw-rewardsDense_goalDetailed.z8..........  	avg. steps:  77.7; avg. score:  8.7 / 10.
tw-another_game.z8..........  	avg. steps: 100.0; avg. score:  5.6 / 8.


#### Evaluating the agent on a test distribution
We will generate 20 test games and evaluate the agent on them.

In [25]:
# You can skip this if you already downloaded the games in the prequisite section.

! seq 1 20 | xargs -n1 -P4 tw-make tw-simple --rewards dense --goal detailed --test --format z8 --output testing_games/ --seed

Global seed: 1
Global seed: 3
Global seed: 4
Global seed: 2
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-ek06H8B7uqoYFVEy.z8
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-E5eLHkaXFk6BSgR1.z8
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-D8gMTlO8cPoEtgZx.z8
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-ekDZtbGXIbO5FKp8.z8
Global seed: 5
Global seed: 6
Global seed: 7
Global seed: 8
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-7KpYUDDdckE0cBqZ.z8
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-o2RVTmrEi6R5T3p0.z8
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-68kvf8x7TBd9Iq0P.z8
Game generated: /content/testing_games/tw-simple-rDense+gDetailed+test-house-GP-KJODI168SvJVFM9x.z8
Global seed: 10
Global seed: 9
Global seed: 11
Global seed: 12
Game generated: /

In [27]:
agent = torch.load('checkpoints/agent_trained_on_multiple_games.pt',weights_only=False)
agent.test()
play(RandomAgent(), "./testing_games/", nb_episodes=20 * 10)
play(agent, "./testing_games/", nb_episodes=20 * 10)  # Averaged over 10 playthroughs for each test game.

./testing_games........................................................................................................................................................................................................  	avg. steps:  99.4; avg. normalized score:  0.5 / 1.
./testing_games........................................................................................................................................................................................................  	avg. steps:  89.2; avg. normalized score:  0.8 / 1.
